In [1]:
import os
import sys
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader

In [2]:
sys.path.append("..")

In [3]:
os.cpu_count()

64

In [4]:
torch.cuda.is_available()

True

In [5]:
torch.cuda.device_count()

4

In [15]:
from carca.data import CARCADataset, load_attrs, load_ctx, load_profiles
from carca.model import CARCA
from carca.train import train, evaluate

In [4]:
# TODO: POSSIBLE ISSUES - CONV1D, DATA LOADING, UNORDERED SEQUENCES, TORCH OPERATIONS (MAYBE SOME PREVENT COMPUTING GRADIENTS)
# MAYBE MAKE OWN CONTEXT (DAY, MONTH, YEAR, etc.)
# !MOST POSSIBLE ISSUE - LACK OF MASKING IN ATTENTION LAYERS!

In [7]:
attrs = load_attrs("video_games")
ctx = load_ctx("video_games")
user_ids, item_ids, profiles = load_profiles("video_games")

In [8]:
n_items = len(item_ids) + 1
n_ctx = next(iter(ctx.values())).shape[0]
n_attrs = attrs.shape[1]

In [9]:
# Hyper-parameters for Games dataset
learning_rate =  0.0001
seq_len = 50
n_blocks = 3
n_heads = 3
dropout_rate = 0.5
l2_reg = 0.0
d_dim = 90
g_dim = 450
residual_sa = True
residual_ca = True
epochs = 800
batch_size = 128
beta1 = 0.9
beta2 = 0.98

In [10]:
train_data = CARCADataset(
    user_ids=user_ids,
    item_ids=item_ids,
    profiles=profiles,
    attrs=attrs,
    ctx=ctx,
    profile_seq_len=seq_len,
    target_seq_len=100,
    mode="train"
)
val_data = CARCADataset(
    user_ids=user_ids,
    item_ids=item_ids,
    profiles=profiles,
    attrs=attrs,
    ctx=ctx,
    profile_seq_len=seq_len,
    target_seq_len=100,
    mode="val"
)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=8)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=8)

In [11]:
model = CARCA(
    n_items=n_items,
    d=d_dim,
    g=g_dim,
    n_ctx=n_ctx,
    n_attrs=n_attrs,
    H=n_heads,
    p=dropout_rate,
    B=n_blocks,
    res_sa=residual_sa,
    res_ca=residual_ca
)

In [12]:
optim = Adam(model.parameters(), lr=learning_rate, weight_decay=l2_reg, betas=(beta1, beta2))

In [13]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cuda:2"
print(f"Using {device} device")
model = model.to(device)

Using cuda:2 device


In [14]:
model = train(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    optim=optim,
    epochs=epochs
)

09:19:56 - Epoch 001: Avg Loss = 1.3943
09:20:01 - Epoch 001: HR = 0.1028, NDCG = 0.0471
09:20:07 - Epoch 002: Avg Loss = 1.3905
09:20:12 - Epoch 002: HR = 0.1096, NDCG = 0.0497
09:20:19 - Epoch 003: Avg Loss = 1.3882
09:20:23 - Epoch 003: HR = 0.0826, NDCG = 0.0375
09:20:30 - Epoch 004: Avg Loss = 1.3857
09:20:35 - Epoch 004: HR = 0.1073, NDCG = 0.0483
09:20:42 - Epoch 005: Avg Loss = 1.3870
09:20:46 - Epoch 005: HR = 0.1027, NDCG = 0.0460
09:20:53 - Epoch 006: Avg Loss = 1.3852
09:20:58 - Epoch 006: HR = 0.1004, NDCG = 0.0459
09:21:05 - Epoch 007: Avg Loss = 1.3834
09:21:09 - Epoch 007: HR = 0.1003, NDCG = 0.0455
09:21:16 - Epoch 008: Avg Loss = 1.3820
09:21:20 - Epoch 008: HR = 0.1194, NDCG = 0.0551
09:21:27 - Epoch 009: Avg Loss = 1.3795
09:21:31 - Epoch 009: HR = 0.1102, NDCG = 0.0502
09:21:37 - Epoch 010: Avg Loss = 1.3766
09:21:42 - Epoch 010: HR = 0.1026, NDCG = 0.0462
09:21:49 - Epoch 011: Avg Loss = 1.3728
09:21:53 - Epoch 011: HR = 0.1059, NDCG = 0.0479
09:22:00 - Epoch 012:

In [16]:
test_data = CARCADataset(
    user_ids=user_ids,
    item_ids=item_ids,
    profiles=profiles,
    attrs=attrs,
    ctx=ctx,
    profile_seq_len=seq_len,
    target_seq_len=100,
    mode="test"
)

test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=8)

In [18]:
evaluate(model, test_loader, device, 10)

(0.5191811681739528, 0.2943886751261299)